In [62]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import os
import time
from collections import defaultdict
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,roc_curve
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
import random
random.seed(1)
from sklearn.linear_model import Lasso

* The symptoms included are as follows:

In [63]:
symptoms = ['intercept',
            'fever',
            'sorethroat',
            'cough',
            'muscle',
            'headache',
            'fatigue',
            'vomit',
            'nausea',
            'diarrhea',
            'chills',
            'sneeze',
            'shortness of breath',
            'phlegm',
            'blockednose',
            'earache',
            'leg pain',
            'runnynose',
            'virus']
aucs_ = defaultdict()

In [64]:
print(symptoms)

['intercept', 'fever', 'sorethroat', 'cough', 'muscle', 'headache', 'fatigue', 'vomit', 'nausea', 'diarrhea', 'chills', 'sneeze', 'shortness of breath', 'phlegm', 'blockednose', 'earache', 'leg pain', 'runnynose', 'virus']


In [65]:
def read_file(filename):
    data = pd.read_csv(filename)
    data['intercept'] = 1
    columns = list(data.columns)
    columns = columns[-1:] + columns[:-1]
    data = data[columns]
#     train_data = data.drop(['virus'],axis =1).as_matrix()
    return data

In [66]:
def read_parameters(filename):
    parameters = pd.read_csv(filename)
    return parameters

#### Get the parameters for the different dataset combinations

In [67]:
directory_ = "./Generated_Parameters_Age/"
with_demographics_ = ['with_demographics','with_demographics_goviral.csv','with_demographics_fluwatch.csv','with_demographics_hongkong.csv','with_demographics_hutterite.csv']
with_demographic_parameters = defaultdict()

In [68]:
def return_parameters(file,parameters_of):
    param = read_parameters(file)
    parameter_dict = defaultdict()
    for i in parameters_of:
        parameter_dict[i] = list(param[i])
    return parameter_dict
    

In [69]:
def get_parameters(dataset_name,parameters):
    return np.array(list(parameters[dataset_name]))

In [70]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [71]:
def get_results(param,sample_points):
    return sigmoid(np.dot(param,sample_points.T)  )

In [72]:
def save_results_for_finding_threshold(filename,dataframe,predicted):
    results = pd.DataFrame()
    results['Actual'] = dataframe['virus']
    results['Predicted'] = predicted
    print(results.head())
    results.to_csv(filename,index = False)

In [73]:
def get_all_datasets(training_data_,training_directory):
    datasets = defaultdict()
    for i in training_data_:
        data = read_file(training_directory+i)
        datasets[i[:-4]] = (data)
    return datasets

In [74]:
def get_all_results(data_dict,param):
    results = defaultdict()
    for i in list(param.keys()):
        data,train = data_dict[i]
        results[i] = get_results(param[i],train)
    return results

In [75]:
def result_statistics(list_):
#     print("Min : ",min(list_))
#     print("Max : ",max(list_))
#     print("Mean : ",np.mean(list_))
#     print("Standard Deviation : ",np.std(list_))
    return min(list_),max(list_)

In [76]:
def return_class(threshold,list_):
    ans = [1 if x >= threshold else 0 for x in list_]
    return ans

def metrics_pred(list1,list2):
    f1 =f1_score(list1,list2)
    precision = precision_score(list1,list2)
    recall = recall_score(list1,list2)
    accuracy = accuracy_score(list1,list2)
    fpr,tpr,threshold = roc_curve(list1,list2)
    auc = metrics.auc(fpr,tpr)
#     print("f1 score : ",f1)
#     print("Precision score : ",precision)
#     print("Recall : ",recall)
#     print("Accuracy : ",accuracy)
#     print("Area under the curve : ",auc)
    return auc

In [77]:
def find_threshold(min_,max_,list1,list2,step_size = 1e-3):
    auc_thresholds = defaultdict()
    value = min_
    while value < max_:
        auc_thresholds[value] = metrics_pred(list1,return_class(value,list2))
        value += step_size
    optimal_threshold = max(auc_thresholds.items(), key=lambda x: x[1]) 
    return optimal_threshold

In [78]:
def get_threshold(pred,true):
    min_,max_ = result_statistics(pred)
    threshold = find_threshold(min_,max_,true,pred)
    return threshold

In [79]:
def return_all_thresholds(results,data,y_true):
    thresholds = defaultdict()
    for i in list(data.keys()):
        print("_____________________")
        min_,max_ = result_statistics(results[i])
        
        threshold = find_threshold(min_,max_,y_true[i],results[i])
        print("Found threshold for : ",i)
        thresholds[i] = threshold
    return thresholds

In [80]:

def test(filename_,param,thresholds_):
    aucs = defaultdict()
    data,train = read_file(filename_)
    for i in list(param.keys()):
        test_results = get_results(param[i],train)
        auc_ = metrics_pred(data['virus'],return_class(thresholds_[i][0],test_results))
        aucs[i] = auc_
    return aucs

In [81]:
def return_final_auc_scores(training_data_,training_directory,filename_,parameters):
    data = get_all_datasets(training_data_)
    results = get_all_results(data,parameters)
    #find the thresholds
    thresholds = return_all_thresholds(results,data)
    #get the auc values
    aucs_= test(filename_,parameters,thresholds)
    return aucs_


In [82]:
def create_dict(dict_):
    temp = []
    for k,v in dict_.items():
        temp.append((k,v))
    return temp
        

In [83]:
results_symp = defaultdict()
results_demo = defaultdict()

#### Get the symptoms

In [84]:
def get_gender(dataframe_):
    df = dataframe_[['male','female']]
    temp = df.apply(lambda x:x.argmax(),axis =1)
    return temp

In [85]:
def get_age(dataframe_):
    df = dataframe_[['age 0-4', 'age 5-15', 'age 16-44', 'age 45-64', 'age 65+']]
    temp = df.apply(lambda x: x.argmax(), axis=1)
    return temp

In [86]:
def get_predictions_all(name,train,only_symp_age,only_symp_gender,param_dict,temp_age,temp_gender,collection_mode = 'clinically_collected',population ='population'):
    results = []
    for i in range(train.shape[0]):
        result = []
        sample_point = train[i,:]
        gender = list(only_symp_gender.iloc[i][:])
        age = list(only_symp_age.iloc[i][:])
        p_data = get_results(param_dict[name],sample_point)
        result.append(p_data)
#         result.append(gender[0]*get_results(param_dict['male'],sample_point))
#         result.append(gender[1]*get_results(param_dict['female'],sample_point))

        result.append(age[0]*get_results(param_dict['age 0-4'],sample_point))
        result.append(age[1]*get_results(param_dict['age 5-15'],sample_point))
        result.append(age[2]*get_results(param_dict['age 16-44'],sample_point))
        result.append(age[3]*get_results(param_dict['age 45-64'],sample_point))
        result.append(age[4]*get_results(param_dict['age 65+'],sample_point))
        
#         p_collection = get_results(param_dict[collection_mode],sample_point)
#         p_gender = get_results(param_dict[temp_gender[i]],sample_point)
#         p_age = get_results(param_dict[temp_age[i]],sample_point)
# #         p_population = get_results(param_dict[population],sample_point)
#         result = [p_data,p_collection,p_gender+p_age]
        results.append(result)
    return results
    

In [87]:
def get_predictions_collection(name,train,param_dict,temp_age,temp_gender,collection_mode = 'clinically_collected',population ='population'):
    results = []
    for i in range(train.shape[0]):
        sample_point = train[i,:]
#         p_data = get_results(param_dict[name],sample_point)
        p_collection = get_results(param_dict[collection_mode],sample_point)
#         p_gender = get_results(param_dict[temp_gender[i]],sample_point)
#         p_age = get_results(param_dict[temp_age[i]],sample_point)
#         p_population = get_results(param_dict[population],sample_point)
        result = [p_collection]
        results.append(result)
    return results
    

In [88]:
def get_predictions_demographic(name,train,param_dict,temp_age,temp_gender,collection_mode = 'clinically_collected',population ='population'):
    results = []
    for i in range(train.shape[0]):
        sample_point = train[i,:]
#         p_data = get_results(param_dict[name],sample_point)
#         p_collection = get_results(param_dict[collection_mode],sample_point)
        p_gender = get_results(param_dict[temp_gender[i]],sample_point)
        p_age = get_results(param_dict[temp_age[i]],sample_point)
#         p_population = get_results(param_dict[population],sample_point)
        result = [p_gender+p_age]
        results.append(result)
    return results
    

In [89]:
def get_predictions_dataset(name,train,param_dict,temp_age,temp_gender,collection_mode = 'clinically_collected',population ='population'):
    results = []
    for i in range(train.shape[0]):
        sample_point = train[i,:]
        p_data = get_results(param_dict[name],sample_point)
# #         p_collection = get_results(param_dict[collection_mode],sample_point)
#         p_gender = get_results(param_dict[temp_gender[i]],sample_point)
#         p_age = get_results(param_dict[temp_age[i]],sample_point)
#         p_population = get_results(param_dict[population],sample_point)
        result = [p_data]
        results.append(result)
    return results

In [90]:
def get_coeff(X,Y):
    lm = linear_model.LogisticRegression()
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
    lm.fit(x_train,y_train)
    y_pred = lm.predict(x_test)
    acc = accuracy_score(y_test,y_pred)
#     print("Accuracy :",acc)
    fpr,tpr,threshold = roc_curve(y_test,y_pred)
    auc_score = metrics.auc(fpr,tpr)
#     print("AUC :",auc_score)
    coefficients = lm.coef_.tolist()[0]
    print("Coefficients : ",coefficients)
    intercept = lm.intercept_.tolist()[0]
    return coefficients,intercept
    

In [91]:
def norm(list_):
    min_ = min(list_)
    max_ = max(list_)
    denom = max_ - min_
    ans = [x-min_/denom for x in list_]
    return ans

In [92]:
COLLECTION_MODE = {'nyumc':'clinically_collected',
                   'goviral':'individually_reported',
                   'fluwatch':'individually_reported',
                   'hongkong': 'health_worker',
                   'hutterite':'health_worker'}

In [93]:
def process_all(training_data_list,training_directory,filename_,parameters,collection_mode = COLLECTION_MODE):
    name_dataset = filename_.split('/')[-1]
    thresholds = defaultdict()
    print(name_dataset)
    data = get_all_datasets(training_data_list,training_directory)
    print("Got the data")
    print("Now finding coefficients for the the datasets!")
    weights = defaultdict()
    for i in data.keys():
        print("Analyzing the dataset : ",i)
        data_ = data[i]
        temp_age = get_age(data_)
        temp_gender = get_gender(data_)
        only_symp_data = data_[symptoms]
        only_symp_gender = data_[['male','female']]
        only_symp_age = data_[['age 0-4','age 5-15','age 16-44','age 45-64','age 65+']]
        only_symp_data.drop('virus',axis = 1,inplace = True)
        train_data_symp = only_symp_data.as_matrix()
        prediction = get_predictions_all(i,train_data_symp,only_symp_age,only_symp_gender,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        y_true = list(data_['virus'])
        coefficient,intercept = get_coeff(prediction,y_true)
        weights[i] = (coefficient,intercept)
        value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
        values = [sigmoid(j) for j in value]
        threshold = get_threshold(values,y_true)
        print("Found threshold for ",i)
        thresholds[i] = threshold[0]
        ans = [(y_true[i],values[i]) for i in range(len(y_true))]
    return weights,thresholds,ans

In [94]:
def process_only_dataset(training_data_list,training_directory,filename_,parameters,collection_mode = COLLECTION_MODE):
    name_dataset = filename_.split('/')[-1]
    thresholds = defaultdict()
    print(name_dataset)
    data = get_all_datasets(training_data_list,training_directory)
    print("Got the data")
    print("Now finding coefficients for the the datasets!")
    weights = defaultdict()
    for i in data.keys():
        print("Analyzing the dataset : ",i)
        data_ = data[i]
        temp_age = get_age(data_)
        temp_gender = get_gender(data_)
        only_symp_data = data_[symptoms]
        only_symp_data.drop('virus',axis = 1,inplace = True)
        train_data_symp = only_symp_data.as_matrix()
        prediction = get_predictions_dataset(i,train_data_symp,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        y_true = list(data_['virus'])
        values = [i[0] for i in prediction]
#         coefficient,intercept = get_coeff(prediction,y_true)
#         weights[i] = (coefficient,intercept)
#         value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
#         values = [sigmoid(j) for j in value]
        threshold = get_threshold(values,y_true)
        print("Found threshold for ",i)
        thresholds[i] = threshold[0]
        ans = [(y_true[i],values[i]) for i in range(len(y_true))]
    return weights,thresholds,ans

In [95]:
def process_collection(training_data_list,training_directory,filename_,parameters,collection_mode = COLLECTION_MODE):
    name_dataset = filename_.split('/')[-1]
    thresholds = defaultdict()
    print(name_dataset)
    data = get_all_datasets(training_data_list,training_directory)
    print("Got the data")
    print("Now finding coefficients for the the datasets!")
    weights = defaultdict()
    for i in data.keys():
        print("Analyzing the dataset : ",i)
        data_ = data[i]
        temp_age = get_age(data_)
        temp_gender = get_gender(data_)
        only_symp_data = data_[symptoms]
        only_symp_data.drop('virus',axis = 1,inplace = True)
        train_data_symp = only_symp_data.as_matrix()
        prediction = get_predictions_collection(i,train_data_symp,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        y_true = list(data_['virus'])
        coefficient,intercept = get_coeff(prediction,y_true)
        weights[i] = (coefficient,intercept)
        value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
        values = [sigmoid(j) for j in value]
        threshold = get_threshold(values,y_true)
        print("Found threshold for ",i)
        thresholds[i] = threshold[0]
        ans = [(y_true[i],values[i]) for i in range(len(y_true))]

    return weights,thresholds,ans

In [96]:
def process_demographic(training_data_list,training_directory,filename_,parameters,collection_mode = COLLECTION_MODE):
    name_dataset = filename_.split('/')[-1]
    thresholds = defaultdict()
    print(name_dataset)
    data = get_all_datasets(training_data_list,training_directory)
    print("Got the data")
    print("Now finding coefficients for the the datasets!")
    weights = defaultdict()
    for i in data.keys():
        print("Analyzing the dataset : ",i)
        data_ = data[i]
        temp_age = get_age(data_)
        temp_gender = get_gender(data_)
        only_symp_data = data_[symptoms]
        only_symp_data.drop('virus',axis = 1,inplace = True)
        train_data_symp = only_symp_data.as_matrix()
        prediction = get_predictions_demographic(i,train_data_symp,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        y_true = list(data_['virus'])
        coefficient,intercept = get_coeff(prediction,y_true)
        weights[i] = (coefficient,intercept)
        value = np.array(np.dot(prediction,np.array(weights[i][0]).T)+weights[i][1])
        values = [sigmoid(j) for j in value]
        threshold = get_threshold(values,y_true)
        print("Found threshold for ",i)
        thresholds[i] = threshold[0]
        ans = [(y_true[i],values[i]) for i in range(len(y_true))]

    return weights,thresholds,ans

In [97]:
def process_test(training_directory,filename_,parameters,weights,thresholds,collection_mode = COLLECTION_MODE):
    aucs_ = defaultdict()
    predictions = defaultdict()
    test_data = get_all_datasets([filename_],training_directory)
    name = filename_.split('.')[0]
    print("Name : ",name) 
    data_ = test_data[name]
    temp_age = get_age(data_)
    temp_gender = get_gender(data_)
    only_symp_data = data_[symptoms]
    only_symp_data.drop('virus',axis = 1,inplace = True)
    y_true = list(data_['virus'])
    train_data_symp = only_symp_data.as_matrix()
    for i in weights.keys():
        print("Using the parameters of : ",i)
        prediction = get_predictions(i,train_data_symp,parameters,temp_age,temp_gender,COLLECTION_MODE[i])
        temp = [i[1:] for i in prediction]
        first = [i[0] for i in prediction]
        prediction = np.array(temp)
        value = np.array(np.dot(temp,np.array(weights[i][0]).T)+weights[i][1]+first)
        values = [sigmoid(i) for i in value]
        predictions[i] = values
    print("Got the predicitions from the different parameters")
    for i in weights.keys():
        auc_ = metrics_pred(y_true,return_class(0.5,predictions[i]))
        aucs_[i] = auc_
        print("Found the auc for ",i)
    return aucs_
#         

#### With demographics

##### Generating the results for HongKong

In [102]:
training_data_goviral = ['goviral.csv']
training_directory = "../../Data/Symptoms_Demo/Balanced_Data/Train/"
filename_ = 'goviral.csv'

In [103]:
cols = [ 'goviral', 'individually_reported', 'health_worker', 'age 0-4', 'age 5-15','age 16-44','age 45-64','age 65+','population']
demo_goviral = return_parameters(directory_+'with_demographics.csv',cols)

In [104]:
print("With demographics!")

With demographics!


##### All the levels

In [105]:
weights_all,thresholds_all,prediction_all = process_all(training_data_goviral,training_directory,filename_,demo_goviral)

goviral.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  goviral
Coefficients :  [-0.02443356372242698, 0.18274747741109207, 0.17654853261702605, -0.7143620738318895, 0.09806200715486992, 0.39024375421341206]
Found threshold for  goviral


In [106]:
thresholds_all

defaultdict(None, {'goviral': 0.46889127123859414})

In [107]:
weights_all['goviral'][0]

[-0.02443356372242698,
 0.18274747741109207,
 0.17654853261702605,
 -0.7143620738318895,
 0.09806200715486992,
 0.39024375421341206]

##### Hongkong

In [112]:
training_data_nyumc = ['fluwatch.csv']
training_directory = "../../Data/Symptoms_Demo/Balanced_Data/"
filename_ = 'fluwatch.csv'

In [113]:
cols = [ 'fluwatch', 'individually_reported', 'health_worker', 'age 0-4', 'age 5-15','age 16-44','age 45-64','age 65+', 'population']
demo_nyumc = return_parameters(directory_+'with_demographics.csv',cols)

In [114]:
weights_all,thresholds_all,prediction_all = process_all(training_data_nyumc,training_directory,filename_,demo_nyumc)

fluwatch.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  fluwatch
Coefficients :  [0.039653287395670124, 0.6801684509430184, 0.7381600925249, 0.8990981423160382, -1.3515325032958443, -0.052516415015758086]
Found threshold for  fluwatch


In [115]:
weights_all['fluwatch'][0]

[0.039653287395670124,
 0.6801684509430184,
 0.7381600925249,
 0.8990981423160382,
 -1.3515325032958443,
 -0.052516415015758086]

###### fluwatch

In [118]:
training_data_nyumc = ['hongkong.csv']
training_directory = "../../Data/Symptoms_Demo/Balanced_Data/"
filename_ = 'hongkong.csv'

In [119]:
cols = [ 'hongkong',  'individually_reported', 'health_worker', 'age 0-4', 'age 5-15','age 16-44','age 45-64','age 65+','population']
demo_nyumc = return_parameters(directory_+'with_demographics.csv',cols)

In [120]:
weights_all,thresholds_all,prediction_all = process_all(training_data_nyumc,training_directory,filename_,demo_nyumc)

hongkong.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  hongkong
Coefficients :  [-3.328736920831443, 5.127491892701865, 2.6257243799801246, -8.206706086057912, -7.837791796360585, -6.7128082428705245]
Found threshold for  hongkong


In [121]:
weights_all['hongkong'][0]

[-3.328736920831443,
 5.127491892701865,
 2.6257243799801246,
 -8.206706086057912,
 -7.837791796360585,
 -6.7128082428705245]

##### Goviral

In [122]:
training_data_nyumc = ['hutterite.csv']
training_directory = "../../Data/Symptoms_Demo/Balanced_Data/"
filename_ = 'hutterite.csv'

In [123]:
cols = ['hutterite',  'individually_reported', 'health_worker', 'age 0-4', 'age 5-15','age 16-44','age 45-64','age 65+', 'population']
demo_nyumc = return_parameters(directory_+'with_demographics.csv',cols)

In [124]:
weights_all,thresholds_all,prediction_all = process_all(training_data_nyumc,training_directory,filename_,demo_nyumc)

hutterite.csv
Got the data
Now finding coefficients for the the datasets!
Analyzing the dataset :  hutterite
Coefficients :  [-0.01520185504703568, 4.508603369786854, 0.7424670155881548, -1.7847774631305153, -2.52941033800328, -2.191276723598417]
Found threshold for  hutterite


In [125]:
weights_all['hutterite'][0]

[-0.01520185504703568,
 4.508603369786854,
 0.7424670155881548,
 -1.7847774631305153,
 -2.52941033800328,
 -2.191276723598417]

In [147]:
def statistics(data):
    male = data.loc[data['male']==1].shape[0]
    female = data.loc[data['female']==1].shape[0]
    age1 = data.loc[data['age 0-4']==1].shape[0]
    age2 = data.loc[data['age 5-15']==1].shape[0]
    age3 = data.loc[data['age 16-44']==1].shape[0]
    age4 = data.loc[data['age 45-64']==1].shape[0]
    age5 = data.loc[data['age 65+']==1].shape[0]
    return (male,female,age1,age2,age3,age4,age5)


In [149]:
data_nyumc = pd.read_csv(training_directory+'goviral.csv')
data_nyumc = data_nyumc.loc[data_nyumc['virus']==1]
nyumc_stats = statistics(data_nyumc)

In [150]:
nyumc_stats

(115, 150, 3, 0, 174, 57, 36)

In [151]:
data_nyumc = pd.read_csv(training_directory+'fluwatch.csv')
data_nyumc = data_nyumc.loc[data_nyumc['virus']==1]
nyumc_stats = statistics(data_nyumc)

In [152]:
nyumc_stats

(252, 314, 41, 79, 149, 230, 67)

In [153]:
data_nyumc = pd.read_csv(training_directory+'hongkong.csv')
data_nyumc = data_nyumc.loc[data_nyumc['virus']==1]
nyumc_stats = statistics(data_nyumc)

In [154]:
nyumc_stats

(598, 873, 45, 367, 651, 295, 75)

In [155]:
data_nyumc = pd.read_csv(training_directory+'hutterite.csv')
data_nyumc = data_nyumc.loc[data_nyumc['virus']==1]
nyumc_stats = statistics(data_nyumc)

In [156]:
nyumc_stats

(289, 498, 83, 256, 256, 113, 31)

In [250]:
predictions = pd.DataFrame()
predictions['Predicted_Value_All'] = [i[1] for i in prediction_all]

In [251]:
predictions['Predicted_Value_Collection'] = [i[1] for i in prediction_collection]

In [252]:
predictions['Predicted_Value_Demographic'] = [i[1] for i in prediction_demographic]

In [253]:
predictions['Predicted_Value_Dataset'] = [i[1] for i in prediction_dataset]

In [254]:
predictions['True'] = [i[0] for i in prediction_all]

In [255]:
predictions[0:10]

,Predicted_Value_All,Predicted_Value_Collection,Predicted_Value_Demographic,Predicted_Value_Dataset,True
0,0.450453,0.455603,0.452410,0.523619,1
1,0.290991,0.290858,0.297370,0.450590,0
2,0.698818,0.678647,0.699158,0.554041,1
3,0.742865,0.741505,0.742480,0.678333,1
4,0.700064,0.695279,0.700145,0.631536,1
5,0.436980,0.458806,0.432685,0.512363,0
6,0.699487,0.678647,0.700034,0.554041,1
7,0.715776,0.727700,0.713543,0.702055,1
8,0.809162,0.794370,0.811930,0.708987,1
9,0.471210,0.491998,0.467149,0.534053,1


In [256]:
predictions['All_Dataset_Specific'] = weights_all['hutterite'][0][0]

In [257]:
predictions['All_Collection_Mode'] = weights_all['hutterite'][0][1]

In [258]:
# predictions['Only_Dataset'] = weights_dataset['goviral'][0][0]

In [259]:
predictions['All_Demographic'] = weights_all['hutterite'][0][2]

In [260]:
predictions['Only_Collection'] = weights_collection['hutterite'][0][0]

In [261]:
predictions['Only_Demographic'] = weights_demographic['hutterite'][0][0]

In [262]:
predictions[0:10]

,Predicted_Value_All,Predicted_Value_Collection,Predicted_Value_Demographic,Predicted_Value_Dataset,True,All_Dataset_Specific,All_Collection_Mode,All_Demographic,Only_Collection,Only_Demographic
0,0.450453,0.455603,0.452410,0.523619,1,-0.249233,0.751449,1.00933,2.68455,1.32303
1,0.290991,0.290858,0.297370,0.450590,0,-0.249233,0.751449,1.00933,2.68455,1.32303
2,0.698818,0.678647,0.699158,0.554041,1,-0.249233,0.751449,1.00933,2.68455,1.32303
3,0.742865,0.741505,0.742480,0.678333,1,-0.249233,0.751449,1.00933,2.68455,1.32303
4,0.700064,0.695279,0.700145,0.631536,1,-0.249233,0.751449,1.00933,2.68455,1.32303
5,0.436980,0.458806,0.432685,0.512363,0,-0.249233,0.751449,1.00933,2.68455,1.32303
6,0.699487,0.678647,0.700034,0.554041,1,-0.249233,0.751449,1.00933,2.68455,1.32303
7,0.715776,0.727700,0.713543,0.702055,1,-0.249233,0.751449,1.00933,2.68455,1.32303
8,0.809162,0.794370,0.811930,0.708987,1,-0.249233,0.751449,1.00933,2.68455,1.32303
9,0.471210,0.491998,0.467149,0.534053,1,-0.249233,0.751449,1.00933,2.68455,1.32303


In [263]:
predictions.to_csv("../Predictions/predictions_hutterite.csv",index=False)